# answers 01: _the case of the dead sand mouse_

__Sean's version__

We don't need any imports. We're starting simple.

I assume that you have the data files [Watson_SuppTable1](http://mcb112.org/w01/Watson_SuppTable1) and [Moriarty_SuppTable2](http://mcb112.org/w01/Moriarty_SuppTable2) downloaded and in the current working directory where you're running Jupyter.

In [1]:
# Good practice: define params, data file names, etc at top of page or script.
#   then it's easy to make changes and re-run
watsonfile   = "Watson_SuppTable1"
moriartyfile = "Moriarty_SuppTable2"

## 1. check that the gene names match

> *Output the names that appear in Watson_SuppTable1 but not Moriarty_SuppTable2, if any*

One of many possible strategies: read each file line by line. Split each line into fields, on whitespace. Field 0 (the first one) is the name. Make a list of the names in each file. Convert the lists to sets, and Python allows you to subtract sets: 


In [3]:
with open(watsonfile) as f1:
    names1 = []                          # initialize an empty list
    for line in f1:
        if line[0] == '#': continue      # skip comment lines
        line   = line.rstrip('\n')       # remove the trailing newline
        fields = line.split()            # split into fields on whitespace
        names1.append(fields[0])         # Add fields[0], the gene name, to the list. 'append' is a method in the list object.

with open(moriartyfile) as f2:
    names2 = []
    for line in f2:
        if line[0] == '#': continue      
        line   = line.rstrip('\n')       
        fields = line.split()            
        names2.append(fields[0])   
    
# A fast pythonic way to get the different elements between two lists: set comparison.
for gene in set(names1) - set(names2):
    print(gene)
    
# Alternatively, the more obvious but slower way:
#  for gene in names1:
#     if gene not in names2: print(gene)

FileNotFoundError: [Errno 2] No such file or directory: 'Watson_SuppTable1'

In [28]:
# May as well do it the other way to see what's going on...
# here's the names in Moriarty that aren't in Watson.
for gene in set(names2) - set(names1):
    print(gene)

SEPT14
SEPT8
SEPT12
SEPT3
MARCH10
SEPT7
SEPT11
MARCH4
MARCH11
SEPT5
SEPT2
MARCH1
MARCH9
MARCH2
MARC2
SEPT4
SEPT1
MARCH5
MARCH6
MARCH7
SEPT6
SEP15
DEC1
SEPT9
MARCH3
MARCH8
MARC1
SEPT10


> *if there's a difference - why?*

What's happened is that Watson et al. imported their data into Microsoft Excel, and by default -- unless you click options that tell it not to -- Excel import will 'helpfully' convert things that look like dates to a standard date format. So the gene "MARCH8" gets converted to "8-Mar". Excel also converts things that look like scientific-notation numbers to a standard number format, so a locus named "2310009E13" would get converted to "2.31E+13". The names in the Moriarty file are uncorrupted.

## 2. explore the data

> _output the five genes with the highest mRNA synthesis rate_

We're going to use the same trick in all three subanswers here. We'll read the rate/tpm numbers into a __dict__, indexed by gene name. We can't sort a dict directly (dicts are unordered). What we need is to sort a separate list of the dict's keys into the order we want, then access the data in the dict in that sorted order. We have a couple of options. 

One way would be to provide a custom sorting function to python's `sort` function that sorts by rate or tpm or whatever using the data in the dict. 

Here we'll use a pythonic shortcut version of that: the `sorted` function can take a dict (among other things) as an argument and return a sorted list of its keys. If you give `sorted` a function as an optional `key` argument, it'll sort on that value instead. So giving it `<dict>.get` -- the dict's own method for retrieving its value -- is sufficient here! A tricksy and useful idiom. `reverse=True` tells `sorted` to sort in descending, rather than the default ascending order.

The point of the exercise is to get used to lists (accessed by index, 0..n-1) and dicts (accessed by keyword), and how to sort them.

In [29]:
with open(moriartyfile) as f2:        
    synth_rate  = {}     # initialize empty dicts
    halflife    = {}    
    for line in f2:
        if line[0] == '#': continue      
        line   = line.rstrip('\n')       
        fields = line.split()            

        synth_rate[fields[0]] = float(fields[1])  # fields[0] is the gene name; our key. fields[1] is synth rate as a STRING.
        halflife[fields[0]]   = float(fields[2])  #   ... we have to convert strings to numbers explicitly: hence float() 


sorted_byrate = sorted(synth_rate, key=synth_rate.get, reverse=True)    # Get a list of gene names (from synth_rate), sorting them on synth_rate's values 
for i in range(5):
    gene = sorted_byrate[i]
    print("{0:15s} {1}".format(gene, synth_rate[gene]))


TLR1            105.2
C12orf54        96.9
SELENBP1        90.2
KRTAP26-1       88.7
GSTA4           69.3


> *output the five genes with the longest mRNA halflife*

Same thing, with the halflife data from Moriarty.

In [30]:
sorted_bylife = sorted(halflife, key=halflife.get, reverse=True)    
for i in range(5):
    gene = sorted_bylife[i]
    print("{0:15s} {1}".format(gene, halflife[gene]))


ATG5            71.8
SPDYE16         64.0
CYB5R2          63.2
LINC00675       63.0
SMLR1           62.3


> *output the five genes with the highest ratio of expression at t=96 hours post-mortem vs. t=0*

Similar to the above, but now we pull in the Watson data, and make a dict of the t=96/t=0 ratio, indexed by gene name.

In [31]:
with open(watsonfile) as f1:
    ratio       = {}     
    for line in f1:
        if line[0] == '#': continue      
        line   = line.rstrip('\n')       
        fields = line.split()            # field [1] = 0hr. [5] = 96hr.

        ratio[fields[0]] = float(fields[5]) / float(fields[1])

sorted_byratio = sorted(ratio, key=ratio.get, reverse=True)    
for i in range(5):
    gene = sorted_byratio[i]
    print("{0:15s} {1:.1f}".format(gene, ratio[gene]))

ATG5            30.4
SPDYE16         25.8
SMLR1           24.5
NOXA1           24.0
CTD-2192J16.20  23.5


The suspicious clue: the most differentially upregulated genes at t=96 are the mRNAs with the longest half lives (the slowest mRNA decay rate).

### 3. Figure out what happened

> _write a script that merges the two data files, line by line, merging them on gene name_

We've already got `synth_rate` and `halflife` data from Moriarty, so let's just read through the Watson file again to merge its lines with the Moriarty numbers.

To write to a file, we open it for writing (`open(outfile, 'w')`), which gives us back a file handle. (Which we have to remember to close.) `print` takes a `file=<filehandle>` option so we can write to any open-for-writing filehandle.

Sometimes you want to print individual fields for a growing line, one at a time, so you don't want `print` to add the automatic newline; the `end=''` argument changes the default end-of-print character from `/n` to nothing (empty string).

Remember the gene name corruption issue. The question said we can ignore genes for which we don't have corresponding lines in both files.



In [32]:
outfile = 'foo.tbl' 
with open(outfile, 'w') as outf:
    # There's some name corruption that we're just going to avoid.  Create
    # a set of the gene names in the ratefile, that we can efficiently
    # check gene names in TPM file against.
    #
    good_genenames = set(synth_rate.keys())


    # Now read the TPM data file (again)
    # Convert its TPM fields to ratios relative to t=0, and append the
    # synthesis rate and halflife fields.
    #
    with open(watsonfile) as f1: 
        for line in f1:
            if line[0] == '#': continue     
            line   = line.rstrip('\n')      
            fields = line.split()           
            if fields[0] not in good_genenames: continue   # Just skip genes with corrupted names,
                                                           # that we can't merge easily to rates.

            tpm   = [float(s) for s in fields[1:]]               # tpm[] is now an array of floats
            ratio = [tpm[i] / tpm[0] for i in range(len(tpm))]   # ratio[] are ratios rel to t=0
    
            print('{:15s} '.format(fields[0]), end='', file=outf)
            for x in ratio[1:]: print('{:6.1f} '.format(x), end='', file=outf)

            print('{:6.1f} '.format(synth_rate[fields[0]]), end='', file=outf) # append synth rate and halflife from Moriarty
            print('{:6.1f} '.format(halflife[fields[0]]),   end='', file=outf)
            print('', file=outf)

Now you have a new file `foo.tbl` in your local directory. The easiest way to explore it is not in python but simply at the command line. 

The key exploration step is to see how "upregulation" systematically correlates with mRNA halflife.

Turns out we can do shell commands in Jupyter Notebook, prefixing them with `!`. 

Do `man sort` and you'll see the manual page for the `sort` command. `-n` means sort numerically. `-r` means reverse, sort from big to small (default is ascending order). `-k5` means sort on the 5th field (whitespace-delimited) of each line, numbering from 1.

In [33]:
! sort -n -r -k5 foo.tbl | head -20

ATG5               1.8    3.3    8.0   30.4    0.6   71.8 
SPDYE16            2.1    3.5    8.5   25.8    1.3   64.0 
SMLR1              2.2    3.5    7.9   24.5    0.9   62.3 
NOXA1              1.7    3.4    7.1   24.0    3.8   60.0 
HAAO               1.9    3.2    7.0   23.5    2.9   60.9 
CTD-2192J16.20     2.1    3.4    8.1   23.5    2.8   58.7 
NSD1               1.9    3.2    7.5   22.8    6.1   55.1 
LINC00675          1.7    2.9    7.2   22.5    0.7   63.0 
CYB5R2             1.8    3.1    6.7   21.8    0.8   63.2 
HESX1              1.9    3.0    7.1   20.6    0.8   59.4 
NFKBID             1.9    2.9    6.3   19.7    3.9   54.8 
XCL1               1.9    3.1    7.0   19.6    1.8   56.3 
TAT                1.9    3.1    6.7   19.6   11.5   52.6 
OR6T1              2.1    3.2    7.3   19.6    5.0   56.6 
LAMB1              2.1    3.3    6.4   19.2    5.0   52.2 
FAM83H             2.0    3.2    6.3   19.2    3.0   51.8 
XPO7               1.8    3.3    7.3   18.8   17.0   55.

(Ignore the write error, if you get one: that's arising from some mishandled detail in Jupyter.)

That's a list of the top 20 most highly upregulated genes at t=96, and now we can see really clearly that there's a strong correlation between the halflife (last column) and the t=96 expression ratio.

### so what happened is...

when Watson killed the sand mouse, new mRNA synthesis stopped, and the existing mRNAs decay away at their rates. The least stable mRNAs go away fastest; the most stable mRNAs go away slowest. RNA-seq measures the _relative_ abundance of each transcript, not the _absolute_ abundance, so it looks like the "expression" of the more stable mRNAs is going up, just because they're going away slower.

Somewhat more subtle is the fact that the effect isn't necessarily monotonic. An mRNA with a moderate halflife will go up (as the least stable mRNAs disappear quickly from the population), then go down (as the moderate-halflife mRNA decays faster than more stable mRNAs).

If Watson et al. had reported their yields from their RNA preps, this could've been a big clue: they should've noticed that the RNA yield was going down relatively quickly with time, because the mRNA population was decaying, which is not really what you'd expect to see if your interpretation is that the sandmouse is running some sort of post-death gene expression program.